In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer,SnowballStemmer
from textblob import TextBlob
from textblob import Word
from nltk.tokenize import word_tokenize
import os

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

from sklearn import model_selection,linear_model,naive_bayes,svm,metrics

In [82]:
Email_Data = pd.read_csv('spam.csv',encoding='latin1')

In [83]:
Email_Data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [84]:
Email_Data = Email_Data[['v1', 'v2']]

In [85]:
Email_Data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [86]:
Email_Data = Email_Data.rename(columns={'v1':"Target",'v2':"Email"})

In [87]:
Email_Data.head()

,Target,Email
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [88]:
#pre processing steps like lower case, stemming and lemmatization
Email_Data["Email"] = Email_Data["Email"].apply(lambda x: x.lower())
stop_word = set(stopwords.words('english'))
Email_Data["Email"] = Email_Data["Email"].apply(lambda x: " ".join(word for word in word_tokenize(x) if word not in stop_word))
stem = PorterStemmer()
lem = WordNetLemmatizer()
# Email_Data["Email"] = Email_Data["Email"].apply(lambda x:" ".join(stem.stem(word) for word in word_tokenize(x)))
Email_Data["Email"] = Email_Data["Email"].apply(lambda x:" ".join(lem.lemmatize(word) for word in word_tokenize(x)))

In [89]:
Email_Data.head()

,Target,Email
0,ham,"go jurong point , crazy .. available bugis n g..."
1,ham,ok lar ... joking wif u oni ...
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor ... u c already say ...
4,ham,"nah n't think go usf , life around though"


In [90]:
train_x,valid_x,train_y,valid_y = train_test_split(Email_Data['Email'],Email_Data['Target'],test_size=0.2)

In [91]:
train_x.size

4457

In [92]:
# TFIDF feature generation for a maximum of 5000 features
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [93]:
train_y

array([0, 0, 1, ..., 0, 0, 0])

In [96]:
tfidf = TfidfVectorizer(analyzer='word',token_pattern='\w{1,}',max_features=5000)
tfidf.fit(Email_Data["Email"])
xtrain_tfidf = tfidf.transform(train_x)
xvalid_tfidf = tfidf.transform(valid_x)

In [95]:
xtrain_tfidf.data

array([0.40606716, 0.55187427, 0.47537252, ..., 0.36839928, 0.27168902,
       0.25023964])

In [102]:
##  Model training
classifier = naive_bayes.MultinomialNB(alpha=0.2)
def train_model(classifier,feature_vector_train,label,valid_y,feature_vector_valid,is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train,label)
    # predict the labels on validation dataset
    prediction = classifier.predict(feature_vector_valid)
    return metrics.accuracy_score(prediction,valid_y)

accuracy = train_model(classifier,xtrain_tfidf,train_y,valid_y,xvalid_tfidf)
print(f"Accuracy {accuracy}")

Accuracy 0.9865470852017937


In [125]:
tex=["click the link to win 100$"]
tex_tfidf = tfidf.transform(tex)


In [126]:
classifier.predict(tex_tfidf)

array([1])